# Google Scholar Profile Scrapper
Test

In [2]:
import GoogleScholarProfile as gsp
import requests
import pandas as pd

file=requests.get('https://raw.githubusercontent.com/restrepo/GoogleScholarProfile/main/tests/test.html').text

l=gsp.GoogleScholarProfile(file)
assert [d.get('cite_id') for d in l]==['8662367354358884167', '5671452052826898641', '']
assert [d.get('cites') for d in l]==['108', '8', '']
assert [d.get('src_journal') for d in l]==['arXiv preprint hep-ph/9906224', 'Physical Review D', '']

#View
pd.DataFrame(l)

,src_title,src_author,src_journal,src_volume,src_number,src_pages,src_year,cite_id,cites
0,Searching for R-parity violation at Run-II of ...,"B Allanach, H Baer, S Banerjee, EL Berger, M C...",arXiv preprint hep-ph/9906224,,,,1999,8662367354358884167,108
1,Dirac neutrino mass generation from a Majorana...,"J Calle, D Restrepo, Ó Zapata",Physical Review D,101,(3),035004,2020,5671452052826898641,8
2,Detailed lecture notes on Quantum Field Theory,D Restrepo,,,,,,,


In [3]:
pd.read_html(file)[0]

,Unnamed: 0,All,Since 2016
0,Citations,116,11
1,h-index,2,2
2,i10-index,1,0


From URL profile

In [4]:
file=requests.get('https://scholar.google.com/citations?user=1sKULCoAAAAJ&hl=en').text
l=gsp.GoogleScholarProfile(file,prefix='')

pd.DataFrame(l)[:2]

,title,author,journal,volume,number,pages,year,cite_id,cites
0,The hunt for new physics at the Large Hadron C...,"P Nath, B Nelson, H Davoudiasl, B Dutta, D Fel...",Nuclear Physics B-Proceedings Supplements,200,,185-417,2010,5836686649283743030,158
1,"Radiative seesaw model: warm dark matter, coll...","DA Sierra, J Kubo, D Suematsu, D Restrepo, O Z...",Physical Review D,79,(1),013011,2009,3332839883472803663,121


With all entries (Download with Firefox)

In [57]:
file=requests.get(
    'https://raw.githubusercontent.com/restrepo/GoogleScholarProfile/main/tests/fcen.html','r').text
l=gsp.GoogleScholarProfile(file,prefix='')

In [58]:
pd.set_option('display.max_rows',3000)

In [59]:
df=pd.DataFrame(l)

In [60]:
df['cites']=df['cites'].apply(lambda n: int(n) if n else 0)
df['year']=df['year'].apply(lambda n: int(n) if n else 0)

In [61]:
df[df['cites']>20].sort_values(['year','cites'],ascending=False).reset_index(drop=True)

,title,author,journal,volume,number,pages,year,cite_id,cites
0,Turtles and tortoises are in trouble,"CB Stanford, JB Iverson, AGJ Rhodin, PP van Di...",Current Biology,30,(12),R721-R735,2020,9341473311343237930,39
1,Magneto-optical transport properties of monola...,"ND Hien, CV Nguyen, NN Hieu, SS Kubakaddi, CA ...",Physical Review B,101,(4),045424,2020,6644591490593251959,32
2,Deciphering\n rhizosphere microbiome assembly ...,"JE Pérez-Jaramillo, M de Hollander, CA Ramírez...",Microbiome,7,(1),1-16,2019,6872814299909410604,43
3,Characterization and study of the antibacteria...,"SMN Gallón, E Alpaslan, M Wang, P Larese-Casan...",Materials Science and Engineering: C,99,,685-695,2019,16967953750371335458,33
4,Simulating\n rewetting events in intermittent ...,"O Shumilova, D Zak, T Datry, D von Schiller, R...",Global change biology,25,(5),1591-1611,2019,8164109467399022544,30
5,Effective\n elimination of fifteen relevant ph...,"EA Serna-Galvis, J Silva-Agredo, AM Botero-Coy...",Science of the total environment,670,,623-632,2019,10980900789503350982,27
6,Seroprevalencia de la enfermedad de Chagas y f...,"R Hoyos, L Pacheco, LA Agudelo, G Zafra, P Bla...",Revista,,,,2019,17966795866318153418,25
7,Investigación cualitativa,"C Trujillo, M Naranjo, K Lomas, M Merlo",Epistemología,,,,2019,9036496479118921767,25
8,Minimal radiative Dirac neutrino mass models,"J Calle, D Restrepo, CE Yaguna, Ó Zapata",Physical Review D,99,(7),075008,2019,3411837993915479956,24
9,Revealing the presence of potential crossings ...,"JF Triana, JL Sanz-Vicario",Physical review letters,122,(6),063603,2019,12136889616511247412,24
